# laptops.csv dataset

In [1]:
#import data set
import pandas as pd
laptops = pd.read_csv('laptops.csv', encoding='Latin-1')


In [2]:
# columns
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Manufacturer              1303 non-null   object
 1   Model Name                1303 non-null   object
 2   Category                  1303 non-null   object
 3   Screen Size               1303 non-null   object
 4   Screen                    1303 non-null   object
 5   CPU                       1303 non-null   object
 6   RAM                       1303 non-null   object
 7    Storage                  1303 non-null   object
 8   GPU                       1303 non-null   object
 9   Operating System          1303 non-null   object
 10  Operating System Version  1133 non-null   object
 11  Weight                    1303 non-null   object
 12  Price (Euros)             1303 non-null   object
dtypes: object(13)
memory usage: 66.2+ KB


## Column names

In [3]:
# Standarize column nanmes
print(laptops.columns)


Index(['Manufacturer', 'Model Name', 'Category', 'Screen Size', 'Screen',
       'CPU', 'RAM', ' Storage', 'GPU', 'Operating System',
       'Operating System Version', 'Weight', 'Price (Euros)'],
      dtype='object')


In [4]:
# cleaning column names: remove white spaces
new_columns = []
for c in laptops.columns:
    clean_c = c.strip()
    new_columns.append(clean_c)
    
laptops.columns = new_columns
print(laptops.columns)

Index(['Manufacturer', 'Model Name', 'Category', 'Screen Size', 'Screen',
       'CPU', 'RAM', 'Storage', 'GPU', 'Operating System',
       'Operating System Version', 'Weight', 'Price (Euros)'],
      dtype='object')


In [5]:
#cleaning: spaces with underscores, remove parenthese, everything in lowercase
def clean_col(col):
    col = col.strip()
    col = col.replace("Operating System", "os")
    col = col.replace(" ","_")
    col = col.replace("(","")
    col = col.replace(")","")
    col = col.lower()
    return col

new_columns = []
for c in laptops.columns:
    clean_c = clean_col(c)
    new_columns.append(clean_c)
    
laptops.columns = new_columns
print(laptops.columns)

Index(['manufacturer', 'model_name', 'category', 'screen_size', 'screen',
       'cpu', 'ram', 'storage', 'gpu', 'os', 'os_version', 'weight',
       'price_euros'],
      dtype='object')


## Data Cleaning

### screen_size, ram -> numeric

In [6]:
# convert string columns with numerica value to numbers

# 1. Explore data && Identify patterns
print(laptops["screen_size"].dtype)
print(laptops["screen_size"].unique())

 
print(laptops["ram"].unique())


object
['13.3"' '15.6"' '15.4"' '14.0"' '12.0"' '11.6"' '17.3"' '10.1"' '13.5"'
 '12.5"' '13.0"' '18.4"' '13.9"' '12.3"' '17.0"' '15.0"' '14.1"' '11.3"']
['8GB' '16GB' '4GB' '2GB' '12GB' '6GB' '32GB' '24GB' '64GB']


In [7]:
# 2.Remove non-numeric characters

laptops["screen_size"] = laptops["screen_size"].str.replace('"','')
laptops["ram"] = laptops["ram"].str.replace('GB','')


print(laptops["screen_size"].unique())
print( laptops["ram"].unique())

['13.3' '15.6' '15.4' '14.0' '12.0' '11.6' '17.3' '10.1' '13.5' '12.5'
 '13.0' '18.4' '13.9' '12.3' '17.0' '15.0' '14.1' '11.3']
['8' '16' '4' '2' '12' '6' '32' '24' '64']


In [8]:
# 3.Convert to numeric

laptops["screen_size"] = laptops["screen_size"].astype(float)
laptops["ram"] = laptops["ram"].astype(int)

print(laptops["screen_size"].dtype)
print(laptops["screen_size"].unique())
print(laptops["ram"].dtype)
print(laptops["ram"].unique())

float64
[13.3 15.6 15.4 14.  12.  11.6 17.3 10.1 13.5 12.5 13.  18.4 13.9 12.3
 17.  15.  14.1 11.3]
int32
[ 8 16  4  2 12  6 32 24 64]


In [9]:
# 4. Rename column (to add information lost in text)

laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)
laptops.rename({"ram": "ram_gb"}, axis=1, inplace=True)

print(laptops.dtypes)



manufacturer           object
model_name             object
category               object
screen_size_inches    float64
screen                 object
cpu                    object
ram_gb                  int32
storage                object
gpu                    object
os                     object
os_version             object
weight                 object
price_euros            object
dtype: object


### Create new column (from other columns)

In [10]:
# Create new column (from other columns)

#print(laptops["gpu"].head().str.split().str[0])

laptops["gpu_manufacturer"] = (laptops["gpu"]
                                       .str.split()
                                       .str[0]
                              )

laptops["cpu_manufacturer"] = (laptops["cpu"]
                                       .str.split()
                                       .str[0]
                               )

print(laptops["gpu_manufacturer"].head())
print(laptops["cpu_manufacturer"].head())

0    Intel
1    Intel
2    Intel
3      AMD
4    Intel
Name: gpu_manufacturer, dtype: object
0    Intel
1    Intel
2    Intel
3    Intel
4    Intel
Name: cpu_manufacturer, dtype: object


In [11]:
print(laptops["cpu_manufacturer"].value_counts())

Intel      1240
AMD          62
Samsung       1
Name: cpu_manufacturer, dtype: int64


### os -> Inconsistent values

In [12]:
# Inconsistent values -> two versions of mac

print(laptops["os"].value_counts())


Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          13
Mac OS          8
Android         2
Name: os, dtype: int64


In [13]:
# Fixing inconsistent values

mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}

laptops["os"] = laptops["os"].map(mapping_dict)

print(laptops["os"].value_counts())


Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          21
Android         2
Name: os, dtype: int64


### Null values

In [14]:
# Null values
print(laptops.isnull().sum())

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight                  0
price_euros             0
gpu_manufacturer        0
cpu_manufacturer        0
dtype: int64


In [15]:
# Removing nulls is not always the best option 

#let's explore
print(laptops["os_version"].value_counts(dropna=False))
print('---')
os_with_null_v = laptops.loc[laptops["os_version"].isnull(),"os"]
print(os_with_null_v.value_counts())

10      1072
NaN      170
7         45
10 S       8
X          8
Name: os_version, dtype: int64
---
No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64


In [16]:
# Conclusions 
# - if there's no OS, obviously there won't be   os_version
# - mac os -> version X
# - chrome -> no version

laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"
laptops.loc[laptops["os"] == "No OS", "os_version"] = "Version Unknown"
laptops.loc[laptops["os"] == "Chrome OS", "os_version"] = "Version Unknown"

value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()
print(value_counts_after)



Linux      62
Android     2
Name: os, dtype: int64


### weight

In [17]:
# Cleaning weight
# 1. Explore
print(laptops["weight"].dtype)
print(laptops['weight'].unique())

# 2. Identify patterns: kg, kgs

object
['1.37kg' '1.34kg' '1.86kg' '1.83kg' '2.1kg' '2.04kg' '1.3kg' '1.6kg'
 '2.2kg' '0.92kg' '1.22kg' '0.98kg' '2.5kg' '1.62kg' '1.91kg' '2.3kg'
 '1.35kg' '1.88kg' '1.89kg' '1.65kg' '2.71kg' '1.2kg' '1.44kg' '2.8kg'
 '2kg' '2.65kg' '2.77kg' '3.2kg' '0.69kg' '1.49kg' '2.4kg' '2.13kg'
 '2.43kg' '1.7kg' '1.4kg' '1.8kg' '1.9kg' '3kg' '1.252kg' '2.7kg' '2.02kg'
 '1.63kg' '1.96kg' '1.21kg' '2.45kg' '1.25kg' '1.5kg' '2.62kg' '1.38kg'
 '1.58kg' '1.85kg' '1.23kg' '1.26kg' '2.16kg' '2.36kg' '2.05kg' '1.32kg'
 '1.75kg' '0.97kg' '2.9kg' '2.56kg' '1.48kg' '1.74kg' '1.1kg' '1.56kg'
 '2.03kg' '1.05kg' '4.4kg' '1.90kg' '1.29kg' '2.0kg' '1.95kg' '2.06kg'
 '1.12kg' '1.42kg' '3.49kg' '3.35kg' '2.23kg' '4.42kg' '2.69kg' '2.37kg'
 '4.7kg' '3.6kg' '2.08kg' '4.3kg' '1.68kg' '1.41kg' '4.14kg' '2.18kg'
 '2.24kg' '2.67kg' '2.14kg' '1.36kg' '2.25kg' '2.15kg' '2.19kg' '2.54kg'
 '3.42kg' '1.28kg' '2.33kg' '1.45kg' '2.79kg' '1.84kg' '2.6kg' '2.26kg'
 '3.25kg' '1.59kg' '1.13kg' '1.78kg' '1.10kg' '1.15kg' '1.27kg' 

In [18]:
# 3.Remove non-numeric characters
laptops["weight"] = laptops["weight"].str.replace('kgs', '')
laptops["weight"] = laptops["weight"].str.replace('kg', '')
print(laptops["weight"])

0       1.37
1       1.34
2       1.86
3       1.83
4       1.37
        ... 
1298     1.8
1299     1.3
1300     1.5
1301    2.19
1302     2.2
Name: weight, Length: 1303, dtype: object


In [19]:
# 4.Convert to numeric

laptops["weight"] = laptops["weight"].astype(float)
print(laptops["weight"].dtypes)

float64


In [20]:
# 5. Rename column (to add information lost in text)

laptops.rename({"weight": "weight_kg"}, axis=1, inplace=True)
print(laptops["weight_kg"].dtypes)

float64


### price

In [21]:
# Price 

#1. Explore
print(laptops["price_euros"].dtype)
# print(laptops['price_euros'].unique())
print(laptops['price_euros'].head())


# 2. Identify patterns: ,



object
0    1339,69
1     898,94
2     575,00
3    2537,45
4    1803,60
Name: price_euros, dtype: object


In [22]:
# 3.Remove non-numeric characters
laptops["price_euros"] = laptops["price_euros"].str.replace(',', '.')

In [23]:
# 4.Convert to numeric
laptops["price_euros"] = laptops["price_euros"].astype(float)

print(laptops['price_euros'].head())
print(laptops["price_euros"].dtype)

0    1339.69
1     898.94
2     575.00
3    2537.45
4    1803.60
Name: price_euros, dtype: float64
float64


### Create: screen resolution


In [24]:

print(laptops.columns)

#1. Explore
print(laptops['screen'].head())

# 2. Identify patterns: 

Index(['manufacturer', 'model_name', 'category', 'screen_size_inches',
       'screen', 'cpu', 'ram_gb', 'storage', 'gpu', 'os', 'os_version',
       'weight_kg', 'price_euros', 'gpu_manufacturer', 'cpu_manufacturer'],
      dtype='object')
0    IPS Panel Retina Display 2560x1600
1                              1440x900
2                     Full HD 1920x1080
3    IPS Panel Retina Display 2880x1800
4    IPS Panel Retina Display 2560x1600
Name: screen, dtype: object


In [25]:
# 3. Extract and create column
# print(laptops["screen"].str.rpartition()[2] )

laptops["screen_resolution"] = laptops["screen"].str.rpartition()[2]  
print(laptops["screen_resolution"].unique())

['2560x1600' '1440x900' '1920x1080' '2880x1800' '1366x768' '2304x1440'
 '3200x1800' '1920x1200' '2256x1504' '3840x2160' '2160x1440' '2560x1440'
 '1600x900' '2736x1824' '2400x1600']


### cpu

In [26]:
#1. Explore

print(laptops['cpu'].head())


# 2. Identify patterns: 

0          Intel Core i5 2.3GHz
1          Intel Core i5 1.8GHz
2    Intel Core i5 7200U 2.5GHz
3          Intel Core i7 2.7GHz
4          Intel Core i5 3.1GHz
Name: cpu, dtype: object


In [27]:
# 3. Extract
print(laptops["cpu"].str.rpartition()[2] )

laptops["cpu_ghz"] = laptops["cpu"].str.rpartition()[2]  
print(laptops["cpu_ghz"].unique())


0       2.3GHz
1       1.8GHz
2       2.5GHz
3       2.7GHz
4       3.1GHz
         ...  
1298    2.5GHz
1299    2.5GHz
1300    1.6GHz
1301    2.5GHz
1302    1.6GHz
Name: 2, Length: 1303, dtype: object
['2.3GHz' '1.8GHz' '2.5GHz' '2.7GHz' '3.1GHz' '3GHz' '2.2GHz' '1.6GHz'
 '2GHz' '2.8GHz' '1.2GHz' '2.9GHz' '2.4GHz' '1.44GHz' '1.5GHz' '1.9GHz'
 '1.1GHz' '2.0GHz' '1.3GHz' '2.6GHz' '3.6GHz' '1.60GHz' '3.2GHz' '1.0GHz'
 '2.1GHz' '0.9GHz' '1.92GHz' '2.50GHz' '2.70GHz']


In [28]:
# 4.Remove non-numeric characters
laptops["cpu_ghz"] = laptops["cpu_ghz"].str.replace('GHz', '')
print(laptops["cpu_ghz"].unique())

['2.3' '1.8' '2.5' '2.7' '3.1' '3' '2.2' '1.6' '2' '2.8' '1.2' '2.9' '2.4'
 '1.44' '1.5' '1.9' '1.1' '2.0' '1.3' '2.6' '3.6' '1.60' '3.2' '1.0' '2.1'
 '0.9' '1.92' '2.50' '2.70']


In [29]:
# 5.Convert to numeric
laptops["cpu_ghz"] = laptops["cpu_ghz"].astype(float)

 
print(laptops["cpu_ghz"].dtype)
print(laptops["cpu_ghz"].unique())

float64
[2.3  1.8  2.5  2.7  3.1  3.   2.2  1.6  2.   2.8  1.2  2.9  2.4  1.44
 1.5  1.9  1.1  1.3  2.6  3.6  3.2  1.   2.1  0.9  1.92]


# SAVE cleaned dataframe

In [35]:
laptops.to_csv('laptops_cleaned.csv', index=False)

# Profiling

In [32]:
import pandas as pd
laptops_cleaned = pd.read_csv('laptops_cleaned.csv', encoding='Latin-1')
laptops_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   manufacturer        1303 non-null   object 
 1   model_name          1303 non-null   object 
 2   category            1303 non-null   object 
 3   screen_size_inches  1303 non-null   float64
 4   screen              1303 non-null   object 
 5   cpu                 1303 non-null   object 
 6   ram_gb              1303 non-null   int64  
 7   storage             1303 non-null   object 
 8   gpu                 1303 non-null   object 
 9   os                  1303 non-null   object 
 10  os_version          1239 non-null   object 
 11  weight_kg           1303 non-null   float64
 12  price_euros         1303 non-null   float64
 13  gpu_manufacturer    1303 non-null   object 
 14  cpu_manufacturer    1303 non-null   object 
 15  screen_resolution   1303 non-null   object 
 16  cpu_gh

In [34]:
import pandas_profiling
laptops_cleaned.profile_report()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]